In [1]:
function evolution(du,u,par,t)
    # this is the function that is taken by the time-integrator
    par_eq, par_inv, h_1, U, M, Flux, Speed_max, Source, c_to_f = par
    χ, ξ = par_eq
    
    if Euler
    c_to_f_direct!(u,(U, M))
    else
    c_to_f!(u,par_inv)
    end
    
    par_mp5 = (h_1, U, M, χ, par_eq, Flux, Speed_max, Source)
    for j ∈ 1:M
        mp5(du,u,par_mp5,j,t)
    end
    return du[:]
end

evolution (generic function with 1 method)

Me make a smaller `dt` evolution for a while in case that the field has to adjust to the sources in a very fast exponential way.

In [ ]:
if !Euler && big_pulse_to_the_right && false
    u_r = zeros(U*M) # Initialization for the second, longer dt step.
    prob_i = ODEProblem(evolution_fast,u_i,(0.0,0.01*tf),par); # solves without changing e and s

    
    #sol = solve(prob_euler_direct,SSPRK33(),dt=dt,saveat=0.01*t); # This, I think, is a TVD method
    sol_i = solve(prob_i,SSPRK33(),dt=0.01*dt,saveat=0.001*tf); 
    u_r = sol_i(0.01*tf)

    field = 1
    #plot(x,reshape(sol(0.),(M,U))[:,field])
    #plot!(x,reshape(sol(t/8),(M,U))[:,field])
    #plot!(x,reshape(sol(t/4),(M,U))[:,field])
    #plot!(x,reshape(sol(t/2),(M,U))[:,field])
    plot(x,reshape(sol_i(tf/1),(M,U))[:,field])

    prob = ODEProblem(evolution_fast,u_r,(0.1*tf,0.5*t),par);
    sol = solve(prob,SSPRK33(),dt=dt,saveat=0.01*t);

    @bell sleep(3)


    sol_i;

    field = 4
    plot(x,reshape(sol_i(0.1*t),(M,U))[:,field])
    u_r = sol_i(0.1*t)
    plot!(x,reshape(u_r,(M,U))[:,field])
end

## Euler equations

We recuperate Euler's equations setting $\chi_1 = 0$ and the dissipative variables to zero. That way the systems decouple. And we get: (the notation here is a bit misleading,
here $\rho \approx e$ Different from $E = (\rho + p) = 4/3 \rho$, since $p = \frac{1}{3}\rho$ and so $s = Ev = 4/3 e v$

$F_e = \frac{4}{3}ργ^2v = s$

$F_s = \frac{4}{3}ρ(γ^2v^2 + \frac{1}{4}) = \frac{1}{3}ργ^2(4 v^2 + \frac{1}{\gamma^2} = 
\frac{1}{3}ργ^2(3v^2 + 1) $

$e = \frac{4}{3}ρ(γ^2 - \frac{1}{4}) = \frac{1}{3}ργ^2(4 - \frac{1}{\gamma^2}) = \frac{1}{3}ργ^2(3 + v^2)$

$s = \frac{4}{3}ργ^2v$

$4\frac{e}{s}v - (3 + v^2) = 0$

$v = 2\frac{e}{s} \pm \sqrt{4(\frac{e}{s})^2 -3}$

$v_- =  \frac{3}{2\frac{e}{s} + \sqrt{4(\frac{e}{s})^2 -3}} \approx \frac{3s}{4e}$


In [2]:
function evolution_euler_direct(du,u,par,t) # here χ₁=0!
    # this is the function that is taken by the time-integrator
    par_eq, par_inv, h_1, U, M, Flux, Speed_max, Source, c_to_f = par
    χ = par_eq
    
    p_direct = (χ, U, M) #these are the parameters for direct
    c_to_f_direct!(u,p_direct)
    #c_to_f!(u,par_inv)
    
    par_mp5 = (h_1, U, M, χ, χ, Flux, Speed_max, Source)
    for j ∈ 1:M
        mp5(du,u,par_mp5,j)
    end
    return du[:]
end
function evolution_euler_direct_fast(du,u,par,t)
    # this is the function that is taken by the time-integrator
    par_eq, par_inv, h, U, M, Flux, Speed_max, Source, c_to_f, par_mem = par
    
    χ, ξ = par_eq
    
    #p_direct = (χ, U, M) #these are the parameters for direct
    #c_to_f_direct!(u,p_direct)
    c_to_f!(u,par_inv)
    
    par_mp5 = (h, U, M, χ, par_eq, Flux, Speed_max, Source, par_mem)
    
    mp5!(du,u,par_mp5,t)
    return du[:]
end

evolution_euler_direct_fast (generic function with 1 method)

In [3]:

prob_euler_direct = ODEProblem(evolution_euler_direct,u_i,(0.0,tf),par);
sol_e = solve(prob_euler_direct,SSPRK33(),dt=dt,saveat=0.01*tf);

LoadError: UndefVarError: tf not defined

In [4]:
#sol = sol_dir_n0;
dsol(t) = sol_nr_n0(t) - sol_dir_n0(t);
reshape(dsol(0.),(M,U))[:,1];

LoadError: UndefVarError: sol_nr_n0 not defined

In [5]:
field = 1
plot(reshape(sol_nr_n0(0.),(M,U))[:,field] .- reshape(sol_dir_n0(0.),(M,U))[:,field])
plot!(reshape(sol_nr_n0(t/8),(M,U))[:,field] .- reshape(sol_dir_n0(t/8),(M,U))[:,field])
plot!(reshape(sol_nr_n0(t/4),(M,U))[:,field] .- reshape(sol_dir_n0(t/4),(M,U))[:,field])
plot!(reshape(sol_nr_n0(t/2.1),(M,U))[:,field] .- reshape(sol_dir_n0(t/2.1),(M,U))[:,field])
#plot!(reshape(sol_nr_n0(t/1.1),(M,U))[:,field] .- reshape(sol_dir_n0(t/1.1),(M,U))[:,field])

LoadError: UndefVarError: sol_nr_n0 not defined

In [6]:
anim = @animate for i ∈ 1:100
    plot(reshape(dsol(t*0.01*i),(M,U))[:,1],label="e", yaxis = ("conserv", (-1e-7,1e-7)))
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,2],label="s")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,3],label="c1")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,4],label="c2")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,5],label="c3")
end

gif(anim, "ff_n0_dif_anim_fps10.gif", fps = 10)

LoadError: LoadError: UndefVarError: @animate not defined
in expression starting at In[6]:1

In [7]:
anim = @animate for i ∈ 1:100
    plot(reshape(dsol(t*0.01*i),(M,U))[:,5+1],label="μ", yaxis = ("fluid", (-1e-8,1e-8)))
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,5+2],label="v")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,5+3],label="X1")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,5+4],label="X2")
    plot!(reshape(dsol(t*0.01*i),(M,U))[:,5+5],label="X3")
end

gif(anim, "ff_n0_dif_anim_fps10.gif", fps = 10)

LoadError: LoadError: UndefVarError: @animate not defined
in expression starting at In[7]:1

In [ ]:
# the initial data
# we give initial data in the fluid variables and then transform to conservatives

u_i=zeros(M*U)
con_0 = view(reshape(u_i,(M,U)),:,1:U÷2)
flu_0 = view(reshape(u_i,(M,U)),:,U÷2+1:U)

if speed_pulse
    

    x0 = 0.0; x1 = 0.2 #x0 = 0.4; x1 = 0.6
    v0 = 0.0; δv = 0.1 

    for i in 1:M
        x[i] = dx*(i-1)
        if x[i] > x0 && x[i] < x1
            flu_0[i,2] = v0 + δv *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250                   #Sz
            flu_0[i,1] = -0.0 # * sin(pi*(x[i] - x0)/(x1-x0))^4 * sin(2*pi*(x[i] - x0)/(x1-x0))     #By
        else
            flu_0[i,2] = v0
        end
    
        flu_0[i,1] = -1.   #
    #flu_0[i,2] = flu_0[i,2] + 0.1  # v
    
    end
f_to_c!(u_i, (χ, U, M)); # populate the conservative variables from the fluid ones
end

if small_pulse_to_the_right

    e0 = 6.; δe = 0.1
    x0 = 0.; x1 = 0.2
    λ = 1. /sqrt(3.)
    for i in 1:M
        x[i] = dx*(i-1)
        if x[i] > x0 && x[i] < x1
            con_0[i,1] = e0 + δe *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250                   #Sz
            con_0[i,2] = λ*(con_0[i,1] - e0)
        else
            con_0[i,1] = e0
            con_0[i,2] = 0.
        end
            flu_0[i,1] = -1.
    end
    par_inv = (χ, tol, 10000, U, M)
    u_i = c_to_f!(u_i,par_inv);
    reshape(u_i,U,M)
end

if big_pulse_to_the_right || big_pulse_negative_I
    

    e0 = 6.; δe = 4.0
    x0 = 0.; x1 = 0.2
    λ = 1. /sqrt(3.)
    for i in 1:M
        x[i] = dx*(i-1)
        if x[i] > x0 && x[i] < x1
            con_0[i,1] = e0 + δe *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250                   #Sz
            con_0[i,2] = λ*(con_0[i,1] - e0)
        else
            con_0[i,1] = e0
            con_0[i,2] = 0.
        end
            flu_0[i,1] = -sqrt(con_0[i,1])/6
    end
    
    if !Euler
        χ_int = [- 1.0; -0.; - 10.0] #primero nos aproximamos a la solución con χ₁=0
        par_inv = (χ, tol, 1000, U, M)
        u_i = c_to_f!(u_i,par_inv);
        χ = [- 1.0; -0.5; - 10.0] #luego buscamos la buena con la semilla anterior
        par_inv = (χ, tol, 1000, U, M)
        u_i = c_to_f!(u_i,par_inv);
        #check!
        if max(abs, f_to_c!(u_i,(χ,U,M)) - u_i) > 0.001
            error("Not converging")
        end
    else
        u_i = c_to_f!(u_i,par_inv);
    end
        reshape(u_i,U,M)
end

if only_diss

    e0 = 6.; δe = 0.0
    x0 = 0.; x1 = 0.2
    x10 = 0.1
    x20 = 0.1
    x30 = 0.1
    #λ = 1. /sqrt(3.)
    for i in 1:M
        x[i] = dx*(i-1)
        if x[i] > x0 && x[i] < x1
            con_0[i,1] = e0
            con_0[i,3] = x10 *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250                   #Sz
            con_0[i,4] = x20 *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250 
            con_0[i,5] = x30 *(x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250 
        else
            con_0[i,1] = e0
        end
        flu_0[i,1] = -1.
    end
        u_i = c_to_f!(u_i,par_inv);
        reshape(u_i,U,M);
end

In [ ]:
function Is_alt(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = ξ[1]χ₀ # χ₀*ξ[1]/χ₁^2 # OK 
    λ = ξ[2]χ₀ #χ₀*ξ[2]/χ₁^2  # OK
    η = ξ[3]χ₀ #χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    γ = (1. - v^2)^(-1//2)
    x1 = 10*γ*T/3*flu[3]
    x2 = flu[4]
    x3 = γ/T*flu[5]
    
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            #(-3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            #(-3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            #(-3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            (-3//10*(γ^2    -1//4)*x1*ξ[1]         + 2γ*v*x2*ξ[2]/T      - μ*v^2*x3*ξ[3])/μ^5;
            (-3//10*(γ^2*v  + 0  )*x1*ξ[1]         + γ*(v^2+1)*x2*ξ[2]/T - μ*v*x3*ξ[3])/μ^5;
            (-3//10*(γ^2*v^2+1//4)*x1*ξ[1]         + 2γ*v*x2*ξ[2]/T      - μ*x3*ξ[3])/μ^5;
            ]
end


In [ ]:
function Flux_cut(u,p)
    χ = p
    f = zeros(length(u))
    #c_to_f(u,χ)    
    #con = view[1:5]
    flu = u[6:end]
    μ = flu[1]  # esto es -μ
    #μ = view(flu,1)
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    f[1] = 0.0 #4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    f[2] = 0.0 #4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    f[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #f[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #f[4] = -(χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v^2+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4)
    f[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(2+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
    f[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    return f[:]
end









In [ ]:
function Flux!(F,u,p)
    χ = p
    #f = zeros(length(u))
    #c_to_f(u,χ)    
    #con = view[1:5]
    flu = u[6:end]
    μ = flu[1]  # esto es -μ
    #μ = view(flu,1)
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    F[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    F[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    F[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #F[4] = -(χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4)
    #F[4] = -(χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v^2+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4)
    F[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(2+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
    F[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1   +3*(1+6γ^2*v^2)*x2       + 3v*x3)/μ^4
    return -F[:]
end


In [ ]:
"""
    Flux(con, p)

Given the fluid variables `flu` and `con` computes the corresponding fluxes.
The value of `con` is used to approximate the next value of it when computed from `flu`. 

As parameters (p) have: 
    p = (flu, χ)
    χ = Function parameters
    con = value of `con` is used to approximate the next value of it when computed from `flu`.

# Examples
```julia-repl
julia> 
χ = [- 1.; - 2.; - 10.]
con = zeros(N)
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu # should vanish
```
"""
function Flux(u,p)
    χ = p
    f = zeros(length(u))
    #c_to_f(u,χ)    
    #con = view[1:5]
    flu = u[6:end]
    μ = flu[1]  # esto es -μ
    #μ = view(flu,1)
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
#    T = (-μ)^(-1//2)
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
#    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q = 10χ₁ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    f[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    f[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    f[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #f[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4
    #f[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4
    f[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v^2+1)*x1     + v*(6γ^2*(2+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
    f[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4
    return -f[:]
end




In [ ]:
"""
    Is!(flu, Is, p)

Given the fluid variables `flu` computes the sources `Is`.

As parameters have: 
    p = decay constants, 

# Examples
```julia-repl
julia> 
Is = zeros(3)
par = (49.735919716217296,16.578639905405765)
I_c = [- 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is!(flu,Is,par) - I_c # should be very small
```
"""
function Is(u,t,par)
    χ, ξ = par #only used to test otherwise are defined below. a is and amplitude to change I from outside
    Is = zeros(10)
    flu = u[6:end]
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= χ[2] # lo hacemos positivo
    if χ₁ < 0
        κ = χ₀*ξ[1]/χ₁^2 # OK 
        λ = χ₀*ξ[2]/χ₁^2  # OK
        η = χ₀*ξ[3]/χ₁^2
    else
        κ = -1.
        λ = -1.
        η = -1.
    end
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1 - v^2)^(-1//2)
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ) + 2γ*v*x2/κ - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ - x3/(γ*η))/μ^4/T;
#            -2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
#            -2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
#            -2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η);
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ);
            0.; 0.; 0.; 0.; 0.
            ]
#    Is[1] = 0.
#    Is[2] = 0.
#    Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
#    Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
#    Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
#    return Is[:]      #(1 - ℯ^(-5. *t))
end


In [ ]:
function Is_dummy(u,t,par)
    Is = zeros(10)
    return Is[:]
end


In [ ]:
"""
    split_u!(u,c,f,N,M) NO FUNCIONA!

split in two matrices the vector u according to the sizes given.
This is a view, not a copy!

    N = number of variables (5x2 here), 
    M = dimension of vector of variables

# Examples
```julia-repl
julia> 
N=10
M=1

u = zeros(N*M)

```
"""
function split_u!(u,c,f,N,M)
    c = view(reshape(u,(M,N)),:,1:N÷2)
    f = view(reshape(u,(M,N)),:,N÷2+1:N)
end


In [ ]:
function F(flu,con,χ)    
    μ = flu[1]  # esto es -μ
    #μ = view(flu,1)
    T = (abs(μ))^(-1/2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    
    γ = (1 - v^2)^(-1//2)
    τ =                        2χ₁ * x3 * T / (γ*μ^3)     + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7//5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2           - 6χ₁*x1/(γ * μ^4 * T)       + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(1-v^2)*x3^2)/(μ^5 * γ^2)
#    Q =                       10χ₁ * x2 * T / μ^3         + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q =                       10χ₁ * x2 * T / μ^3         + 168χ₂ * x2 * (x1 + x3/γ)/(γ * μ^5)

    A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)      3v*(6γ^2 - 1)           3v^2 ;
                       v*(6γ^2 - 1)      (6γ^2*(1 + 2v^2) - 1)   v*(v^2 + 2) ;
                       (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]

    e = 4//3*ρ*(γ^2 - 1//4) + 2v*γ*Q         + v^2*τ
    s = 4//3*ρ*γ^2*v        + (v^2+1//1)γ*Q + v*τ;

    Y1 = χ₁*γ/T/μ^3 *(6γ^2 - 3//1)
    #Y2 = 3χ₁*γ/T/μ^3 *v*(6γ^2 - 1//1)
    Y2 = χ₁*γ/T/μ^3 *v*(6γ^2 - 1//1)
    #Y3 = 3χ₁*γ/T/μ^3 *(6γ^2*v^2 + 1//1)
    Y3 = χ₁*γ/T/μ^3 *(6γ^2*v^2 + 1//1)
    
    return  [- con[1] + 4//3* ρ*(γ^2 - 1//4) + 2//1* Q*γ*v + τ*v^2;
             - con[2] + 4//3* ρ*γ^2*v + Q*γ*(v^2 + 1//1) + τ*v;
             - con[3] + A[1,1]*x1 + A[1,2]*x2 + A[1,3]*x3 + Y1;
             - con[4] + A[2,1]*x1 + A[2,2]*x2 + A[2,3]*x3 + Y2;
             - con[5] + A[3,1]*x1 + A[3,2]*x2 + A[3,3]*x3 + Y3]
end


@variables f[1:5], c[1:5], p[1:3], r[1:5]

JS = Symbolics.jacobian(F(f,c,p),f);
J_exp = Symbolics.build_function(JS, f, c, p);
Jac = eval(J_exp[1]);